# Prototype for the Financial Advisor Template - Interpretable Strategy Engine

In [14]:
# prototype tickers: 2 “good” + 2 “fails”
syms = ["MCD", "TMO", "COP", "PFE"]

	## 1 Data ingestion & cleaning

In [15]:
!python ../src/ingestion/fetch_data.py --tickers MCD TMO COP PFE

Saved data for MCD to c:\FinalProject\src\ingestion\..\..\data\raw/MCD.csv
Saved data for TMO to c:\FinalProject\src\ingestion\..\..\data\raw/TMO.csv
Saved data for COP to c:\FinalProject\src\ingestion\..\..\data\raw/COP.csv
Saved data for PFE to c:\FinalProject\src\ingestion\..\..\data\raw/PFE.csv



[                       0%                       ]
[**********************50%                       ]  2 of 4 completed
[**********************75%***********            ]  3 of 4 completed
[*********************100%***********************]  4 of 4 completed


In [16]:
!python ../src/ingestion/clean_data.py --syms MCD TMO COP PFE


Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\MCD.csv (2726 rows)
Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\TMO.csv (2726 rows)
Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\COP.csv (2726 rows)
Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\PFE.csv (2726 rows)


c:\FinalProject\src\ingestion\clean_data.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)                # propagate last good
c:\FinalProject\src\ingestion\clean_data.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)                # propagate last good
c:\FinalProject\src\ingestion\clean_data.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)                # propagate last good
c:\FinalProject\src\ingestion\clean_data.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill',

In [17]:
!python ../src/features/compute_features.py --syms MCD TMO COP PFE


→ Features for MCD: 2677 rows, 19 cols
→ Features for TMO: 2677 rows, 19 cols
→ Features for COP: 2677 rows, 19 cols
→ Features for PFE: 2677 rows, 19 cols


In [18]:
!python ../src/models/train_models.py --mode holdout --syms MCD TMO COP PFE


→ Saved COP.json (3 total entries)
→ Saved MCD.json (3 total entries)
→ Saved PFE.json (3 total entries)
→ Saved TMO.json (3 total entries)


c:\FinalProject\src\models\train_models.py:119: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'timestamp': datetime.utcnow().isoformat() + 'Z'
c:\FinalProject\src\models\train_models.py:119: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'timestamp': datetime.utcnow().isoformat() + 'Z'
c:\FinalProject\src\models\train_models.py:119: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'timestamp': datetime.utcnow().isoformat() + 'Z'
c:\FinalProject\src\models\train_models.py:119: DeprecationWarning: datetime.datetime.utcnow()

In [19]:
!python ../src/models/grid_tree_search.py --syms MCD TMO COP PFE


√ COP  cv_acc=-inf  saved→c:\FinalProject\src\models/../../data/strategies/COP_grid.json
√ MCD  cv_acc=0.5048  saved→c:\FinalProject\src\models/../../data/strategies/MCD_grid.json
√ PFE  cv_acc=0.4231  saved→c:\FinalProject\src\models/../../data/strategies/PFE_grid.json
√ TMO  cv_acc=0.5484  saved→c:\FinalProject\src\models/../../data/strategies/TMO_grid.json


In [20]:
!python ../src/models/train_rulefit.py --syms MCD TMO COP PFE


Starting RuleFit training...
Using 4 cores for parallel processing...
√ COP  acc=0.507  rules=122  time=62.32s
√ MCD  acc=0.494  rules=121  time=52.34s
√ PFE  acc=0.532  rules=122  time=30.2s
√ TMO  acc=0.539  rules=122  time=37.82s



Training RuleFit models: 100%|██████████| 4/4 [01:04<00:00, 16.15s/symbol]


In [21]:
!python ../src/models/xgb_surrogate.py --syms MCD TMO COP PFE


√ COP: 5 windows saved
√ MCD: 5 windows saved
√ PFE: 5 windows saved
√ TMO: 5 windows saved


In [26]:
!python ../backtest_run.py --syms MCD TMO COP PFE



=== DT BACK-TEST RESULTS ===

    Total Return [%] Sharpe Ratio Max Drawdown [%]
MCD       384.003202     1.193072        20.013062
TMO       919.123533     1.338137         17.20898
COP       634.494042     0.879463        56.322498
PFE       186.689349     0.740888        32.984734

MCD – Decision Tree rules (first 12 lines):
|--- ret_1d <= 0.02
|   |--- adx_14 <= 52.01
|   |   |--- macd_sig <= -2.60
|   |   |   |--- class: 1
|   |   |--- macd_sig >  -2.60
|   |   |   |--- roc_10 <= 1.30
|   |   |   |   |--- class: 1
|   |   |   |--- roc_10 >  1.30
|   |   |   |   |--- class: 0
|   |--- adx_14 >  52.01
|   |   |--- class: 1
|--- ret_1d >  0.02

TMO – Decision Tree rules (first 12 lines):
|--- bb_width <= 0.05
|   |--- atr_14 <= 2.25
|   |   |--- macd_hist <= -0.24
|   |   |   |--- class: 0
|   |   |--- macd_hist >  -0.24
|   |   |   |--- class: 1
|   |--- atr_14 >  2.25
|   |   |--- bb_width <= 0.04
|   |   |   |--- class: 1
|   |   |--- bb_width >  0.04
|   |   |   |--- class: 1
|-

In [ ]:
import importlib
import pandas as pd

importlib.reload(bt)          # pick up any edits
symbols = ["MCD", "TMO", "COP", "PFE"]
stats    = bt.run_backtests(symbols)    # dict tag→DataFrame

# build ROI table (Total Return %)
roi = pd.concat({tag: df["Total Return [%]"] for tag, df in stats.items()}, axis=1)
roi.columns = roi.columns.str.upper()
roi = roi.round(2).sort_index()

display(
    roi.style
       .format("{:+.2f}%")
       .background_gradient(cmap="RdYlGn", axis=None)
       .set_caption("Total Return (%) across engines")
)

ImportError: attempted relative import with no known parent package